In [1]:
# from dotenv import load_dotenv
# load_dotenv()
# GROQ_API_KEY=os.environ.get('GROQ_API_KEY')


# # pip install -U 'crewai[tools]' mdpdf


In [1]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from openai import OpenAI
from langchain_groq import ChatGroq


/Users/aryanraj/Developer/Crews-AI/.venv/lib/python3.12/site-packages/langchain/agents/json_chat/base.py:22: SyntaxWarning: invalid escape sequence '\ '
  """Create an agent that uses JSON to format its logic, build for Chat Models.
/Users/aryanraj/Developer/Crews-AI/.venv/lib/python3.12/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/aryanraj/Developer/Crews-AI/.venv/lib/python3.12/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/aryanraj/Developer/Crews-AI/.venv/lib/python3.12/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [2]:
llm =ChatGroq(
    groq_api_base="https://api.groq.com/openai/v1", # https://api.openai.com/v1 
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="mixtral-8x7b-32768" 
)

In [4]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

In [5]:

llm = ChatGroq(temperature=0.4, groq_api_key='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', model_name="mixtral-8x7b-32768")



In [6]:
from langchain.agents import  Tool
template = """This is a conversation between a human and ai agent:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  
)

In [7]:
search =  DuckDuckGoSearchRun()

tool_use = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )

]

tool_use_1 = [
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )
]

In [8]:
email_author = Agent(
    role='Professional Email Author',
    goal='Craft concise and engaging emails',
    backstory='Experienced in writing impactful marketing emails.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use
)
marketing_strategist = Agent(
    role='Marketing Strategist',
    goal='Lead the team in creating effective cold emails',
    backstory='A seasoned Chief Marketing Officer with a keen eye for standout marketing content.',
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools=tool_use_1
)

content_specialist = Agent(
    role='Content Specialist',
    goal='Critique and refine email content',
    backstory='A professional copywriter with a wealth of experience in persuasive writing.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use_1
    )

     

In [9]:
email_task = Task(
    description='''1. Generate two distinct variations of a cold email promoting a video editing solution.
    2. Evaluate the written emails for their effectiveness and engagement.
    3. Scrutinize the emails for grammatical correctness and clarity.
    4. Adjust the emails to align with best practices for cold outreach. Consider the feedback
    provided to the marketing_strategist.
    5. Revise the emails based on all feedback, creating two final versions.''',
    expected_output='Two final versions of the cold email after considering all feedback.',
    agent=marketing_strategist 
)

In [11]:
import sys

# Redirecting stdout to a file
sys.stdout = open('output_file.txt', 'w')

email_crew = Crew(
    agents=[email_author, marketing_strategist, content_specialist],
    tasks=[email_task],
    verbose=True,
    process=Process.sequential
)

# Execution Flow
print("Crew: Working on Email Task")
emails_output = email_crew.kickoff()

2024-04-17 08:58:14,001 - 8218344128 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed
